In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
import random

from models.resnet import *
from wrapper import TestWrapper, ModelWrapper
import matplotlib.pyplot as plt
from advertorch.attacks import CarliniWagnerL2Attack, L2PGDAttack, L1PGDAttack, LinfPGDAttack, SparseL1DescentAttack
from attack import PGDAttack

import foolbox
import foolbox.attacks as fa
import numpy as np

In [2]:
transform_test = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=2)

In [3]:
class Evaluation:
    def __init__(self, model):
        self.correct = 0
        self.total = 0
        self.model = model
        self._nll = 0

    def evaluate(self, inputs, targets):
        with torch.no_grad():
            outputs = self.model(inputs)
            likelihood, predicted = outputs.max(1)
            self._nll += -torch.log(likelihood).sum().item()

            self.total += targets.size(0)
            self.correct += predicted.eq(targets).sum().item()
        
    @property
    def dr(self):
        return 1. * self.correct / self.total
    

In [5]:
from models.resnet import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
# MIMO setting
ensembles = 3
net = ResNet18(ensembles,30).to(device)

net = torch.nn.DataParallel(net)
cudnn.benchmark = True

assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('checkpoint/ckpt_mm_mnist_25.pth')
net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']
optimizer = checkpoint['optimizer']

criterion = nn.CrossEntropyLoss()

net = TestWrapper(net, ensembles)
# optimizer = optim.Adam(net.parameters(), lr=0.01)
# attack = CarliniWagnerL2Attack(net, num_classes=10)
# attack = PGDAttack(net, alpha=[0.125], epsilon=[0.5], norm=[2], max_iteration=40, random_start=True)
# attack = PGDAttack(net, alpha=[2/255], epsilon=[8/255], norm=[float('inf')], max_iteration=40, random_start=True)
# attack = PGDAttack(net, alpha=[3.], epsilon=[12.], norm=[1], max_iteration=40, random_start=True)
# attack = LinfPGDAttack(net, eps=0.3, nb_iter=100, eps_iter=0.01, rand_init=True)
# attack = L2PGDAttack(net, eps=2., nb_iter=200, eps_iter=0.1, rand_init=True)
# attack = SparseL1DescentAttack(net, eps=12., nb_iter=40, eps_iter=1., l1_sparsity=0.99, rand_init=True)
# attack = CarliniWagnerL2Attack(net, num_classes=10)
net.eval()
evaluation = Evaluation(net)
adv_e = Evaluation(net)

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint/ckpt_mm_mnist_25.pth'